## Environment setup

In [ ]:
!python3.7 -m pip install boto3;

In [ ]:
!python3.7 -m pip install awscli;

## Parameters configuration

In [4]:
import boto3
import os

account = boto3.client('sts').get_caller_identity().get('Account')

AWS_REGION = "us-east-1"

# Configuration for the IoT Thing: Certificate file name, and core name
CERT_PEM_OUTFILE="mli.cert.pem"
PUBLIC_KEY_OUTFILE="mli.public.key" 
PRIVATE_KEY_OUTFILE="mli.private.key" 
CORENAME="greengrass_ml_{}".format(AWS_REGION)

# Temporary workspace
WORKSPACE_FOLDER="./work"
GREENGRASS_WORK_FOLDER=os.path.join(WORKSPACE_FOLDER,"greengrass")

# S3 Bucket for the ML model
ML_S3_BUCKET="{}-greengrass-{}".format(account,AWS_REGION)

# S3 Bucket for the Cloudformation functions
CFN_S3_BUCKET="{}-cloudformation-{}".format(account,AWS_REGION)
CFN_STACK_NAME="greengrass-mli-accelerator"

In [11]:
![ -d {WORKSPACE_FOLDER} ] && rm -r {WORKSPACE_FOLDER} && echo "old {WORKSPACE_FOLDER} removed"
!mkdir -p {WORKSPACE_FOLDER} && echo "{WORKSPACE_FOLDER} created"
!mkdir -p {GREENGRASS_WORK_FOLDER} \
&& mkdir -p {GREENGRASS_WORK_FOLDER}/certs \
&& mkdir -p {GREENGRASS_WORK_FOLDER}/config \
&& echo "{GREENGRASS_WORK_FOLDER} created"

old ./work removed
./work created
./work/greengrass created


## Download the pre-trained model

In [ ]:
!wget -nv http://data.mxnet.io/models/imagenet/inception-bn/Inception-BN-symbol.json --directory-prefix {WORKSPACE_FOLDER}
!wget -nv http://data.mxnet.io/mxnet/models/imagenet/synset.txt --directory-prefix {WORKSPACE_FOLDER}
!wget -nv http://data.mxnet.io/models/imagenet/inception-bn/Inception-BN-0126.params -O {WORKSPACE_FOLDER}/Inception-BN-0000.params

In [ ]:
from zipfile import ZipFile
import os

# create a ZipFile object
zipObj = ZipFile(os.path.join(WORKSPACE_FOLDER, "inception-bn.zip"), 'w')
# Add multiple files to the zip
zipObj.write(os.path.join(WORKSPACE_FOLDER,"Inception-BN-symbol.json"))
zipObj.write(os.path.join(WORKSPACE_FOLDER,"synset.txt"))
zipObj.write(os.path.join(WORKSPACE_FOLDER,"Inception-BN-0000.params"))
# close the Zip File
zipObj.close()

In [ ]:
# Upload to S3 bucket
import boto3
import os
s3 = boto3.resource('s3')
if not s3.Bucket(ML_S3_BUCKET) in s3.buckets.all():
    s3.create_bucket(Bucket=ML_S3_BUCKET)

s3_client = boto3.client('s3')
s3_client.upload_file( os.path.join(WORKSPACE_FOLDER, "inception-bn.zip"), ML_S3_BUCKET, "inception-bn.zip")

ML_S3_BUCKET_URI="s3://{}/{}".format(ML_S3_BUCKET, "inception-bn.zip")

## Create the credential for the AWS IoT Thing

In [ ]:
iotClient = boto3.client('iot', region_name=AWS_REGION)
from botocore.exceptions import ClientError
try:
    response = iotClient.create_keys_and_certificate(
        setAsActive=True
    )
except ClientError as e:
    if(e.response["Error"]["Code"]=="AccessDeniedException"):
        print("Missing permission. Please add \niot:CreateKeysAndCertificate on resource: *\n to the instance IAM role\n")
    raise e

In [ ]:
import os
CERTIFICATE_ID=response.get("certificateId")
CERT_FOLDER = os.path.join(GREENGRASS_WORK_FOLDER, "certs/")
if not os.path.exists(CERT_FOLDER):
    os.makedirs(CERT_FOLDER)                    

try:
    with open(os.path.join(CERT_FOLDER, CERT_PEM_OUTFILE), 'w') as the_file:
        the_file.write(response.get("certificatePem"))
    with open(os.path.join(CERT_FOLDER, PUBLIC_KEY_OUTFILE), 'w') as the_file:
        the_file.write(response.get("keyPair").get("PublicKey"))
    with open(os.path.join(CERT_FOLDER, PRIVATE_KEY_OUTFILE), 'w') as the_file:
        the_file.write(response.get("keyPair").get("PrivateKey"))
except IOError as e:
    print("Error creating certificate files")
    raise e

## Uses the Cloudformation to generate the Greengrass Core

In [ ]:
import boto3
s3 = boto3.resource('s3')
if not s3.Bucket(CFN_S3_BUCKET) in s3.buckets.all():
    s3.create_bucket(Bucket=CFN_S3_BUCKET)

In [ ]:
# Check IAM permission
import boto3
from botocore.exceptions import ClientError

try:
    cloudformation = boto3.resource('cloudformation', region_name=AWS_REGION)
    stack = cloudformation.Stack(CFN_STACK_NAME)
    stack.description
except ClientError as e:
    if(e.response["Error"]["Code"]=="ValidationError"):
        print("Stack name {} does not exist, continue".format(CFN_STACK_NAME))
    elif(e.response["Error"]["Code"]=="AccessDenied"):
        print("Missing permission. Please add the following IAM Policy\n\n \
              resource: arn:aws:cloudformation:{}:{}:stack/{}/*\n\n \
              cloudformation:DescribeStacks \n \
              cloudformation:CreateChangeSet \n \
              to IAM role:\n{}\n".format(region,account,CFN_STACK_NAME,role))
        raise e
    else:
        raise e

In [ ]:
![ -e {WORKSPACE_FOLDER}/*-OUTPUT.yaml ] && rm {WORKSPACE_FOLDER}/*-OUTPUT.yaml
!aws cloudformation package \
--region {AWS_REGION} \
--template-file aws-iot-greengrass-accelerators/accelerators/machine_learning_inference/cfn/mli_accelerator_s3_models-INPUT.cfn.yaml \
--s3-bucket {CFN_S3_BUCKET} \
--output-template-file {WORKSPACE_FOLDER}/mli_accelerator_s3_models-OUTPUT.yaml


In [ ]:
!aws cloudformation deploy \
  --region {AWS_REGION} \
  --stack-name {CFN_STACK_NAME} \
  --template-file {WORKSPACE_FOLDER}/mli_accelerator_s3_models-OUTPUT.yaml \
  --capabilities CAPABILITY_NAMED_IAM \
  --parameter-overrides \
    CoreName={CORENAME} \
    CertIdParam={CERTIFICATE_ID} \
    ModelS3Uri={ML_S3_BUCKET_URI} 

In [ ]:
print("...waiting for stack {} to be ready...".format(CFN_STACK_NAME))
client = boto3.client('cloudformation', region_name=AWS_REGION)
waiter = client.get_waiter('stack_create_complete')
waiter.wait(StackName=CFN_STACK_NAME)

## Create the Greengrass config and package with the credential files

In [12]:
# Download the Amazon ROOT CA cert into the folder
!wget -o {GREENGRASS_WORK_FOLDER}/certs/AmazonRootCA1.pem https://www.amazontrust.com/repository/AmazonRootCA1.pem

In [20]:
# from operator import itemgetter 
import boto3
import json

cfn_client = boto3.client('cloudformation', region_name=AWS_REGION)
response = cfn_client.describe_stacks(StackName=CFN_STACK_NAME)
stacks = response.get("Stacks")

greengrass_config={}

for stack in stacks:
    if stack["Outputs"]:
        for output in stack["Outputs"]:
            if output["OutputKey"] == "GreengrassConfig":
                greengrass_config = output["OutputValue"]
                greengrass_config = json.loads(greengrass_config)
    
# Update the credential file name    
greengrass_config["crypto"]["certificatePath"] = "file:////greengrass/certs/{}".format(CERT_PEM_OUTFILE)
greengrass_config["crypto"]["privateKeyPath"] = "file:////greengrass/certs/{}".format(PRIVATE_KEY_OUTFILE)

try:
    GREENGRASS_CONFIG_FOLDER = os.path.join(GREENGRASS_WORK_FOLDER, "config/")
    if not os.path.exists(GREENGRASS_CONFIG_FOLDER):
        os.makedirs(GREENGRASS_CONFIG_FOLDER)                    
    with open(os.path.join(GREENGRASS_CONFIG_FOLDER, "config.json"), 'w') as the_file:
        the_file.write(json.dumps(greengrass_config))
except IOError as e:
    print("Error creating Greengrass config file")
    raise e

In [29]:
from zipfile import ZipFile
import os

def archive_all_subfolder(zipfile, folder):
    """
    'zipfile' is the file name for the archive with a full path
    'folder' is root of the folder which all files to be added to the zipfile
    """
    with ZipFile(zipfile, 'w') as zip:    
        for root, dirs, files in os.walk(folder):
            if(files):
                for file in files:
                    f = os.path.join(root,file)
                    print ("archiving file {}".format(f))
                    zip.write(f, os.path.relpath(f, root))

archive_all_subfolder(os.path.join(WORKSPACE_FOLDER, "greengrass-config.zip"), GREENGRASS_WORK_FOLDER)

archiving file ./work/greengrass/certs/AmazonRootCA1.pem
archiving file ./work/greengrass/config/config.json


Download the `greengrass-config.zip` to your local computer, and upload to the Greengrass EC2